In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import Levenshtein

input_string=str(input("Enter the product : "))
final_string=input_string.replace(" ","%20")
URL1  = "https://www.amazon.in/s?k=" + final_string
URL2 = "https://www.flipkart.com/search?q=" + final_string

# Headers for request
HEADERS = ({'User-Agent':' ', 'Accept-Language': 'en-US, en;q=0.5'})

In [92]:
# HTTP Request
webpage1 = requests.get(URL1, headers=HEADERS)
webpage2 = requests.get(URL2, headers=HEADERS)

In [93]:
# Soup Object containiang all data
soup1 = BeautifulSoup(webpage1.content, "html.parser")
soup2 = BeautifulSoup(webpage2.content, "html.parser")

In [94]:
def find_best_match(name, dictionary_list):
    best_match = None
    best_distance = float('inf')
    
    for dictionary in dictionary_list:
        dictionary_name = dictionary["item_name"]
        distance = Levenshtein.distance(name, dictionary_name)
        if distance < best_distance:
            best_distance = distance
            best_match = dictionary

    return best_match

def remove_quotes_from_url(url):
    if url.startswith('"') or url.startswith("'"):
        url = url[1:]
    if url.endswith('"') or url.endswith("'"):
        url = url[:-1]
    return url



In [95]:

#---------amazon---------
amz_name_list=[]
amz_product_link_list=[]
try:
    amz_product_name = soup1.find_all("span", attrs={"class":"a-size-medium a-color-base a-text-normal"})
    if(len(amz_product_name)<=3):
        raise Exception('not valid')
except Exception as e:
    amz_product_name=soup1.find_all('span',attrs={'class':"a-size-base-plus a-color-base a-text-normal"})

try:
    amz_product_link = soup1.find_all('a',attrs={"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    if len(amz_product_link)==0:
        raise Exception ('Empty')
except Exception as e:
    amz_product_link=soup1.find_all('a',attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})


for i in range (0,len(amz_product_name)):
    amz_name_list.append(str(amz_product_name[i].text)[:50])
    amz_product_link_list.append("https://www.amazon.in"+(amz_product_link[i].get('href')))


item_on_amz_page=[]
for i in range(0,len(amz_name_list)):
    item = {"item_name": amz_name_list[i], "item_link": amz_product_link_list[i]}
    item_on_amz_page.append(item)

best_match = find_best_match(input_string, item_on_amz_page)
best_match = find_best_match(input_string, item_on_amz_page).get('item_link')
amazon_result_url = remove_quotes_from_url(best_match)

#---------flipkart--------

flip_name_list=[]
flip_product_link_list=[]
try:
    flip_product_name=soup2.find_all("div", attrs={"class":"_4rR01T"})
    if len(flip_product_name)==0:
        raise Exception ('Invalid')
except:
    flip_product_name=soup2.find_all("a", attrs={"class":"s1Q9rs"})

try:
    flip_product_link=soup2.find_all("a",attrs={"class":"_1fQZEK"})
    if len(flip_product_link)==0:
        raise Exception ('Invalid')
except:
    flip_product_link=soup2.find_all('a',attrs={"class":'s1Q9rs'})

for i in range(0,len(flip_product_name)):
    flip_name_list.append(str(flip_product_name[i].text)[:50])
    flip_product_link_list.append("https://flipkart.com"+(flip_product_link[i].get('href')))

item_on_flip_page=[]
for i in range(0,len(flip_name_list)):
    item={'item_name':flip_name_list[i],'item_link':flip_product_link_list[i]}
    item_on_flip_page.append(item)
best_match=find_best_match(input_string,item_on_flip_page).get('item_link')
flipkart_result_url=remove_quotes_from_url(best_match)

    

In [ ]:
new_webpage1=requests.get(amazon_result_url,headers=HEADERS)
new_webpage2=requests.get(flipkart_result_url,headers=HEADERS)

new_soup1=BeautifulSoup(new_webpage1.content,'html.parser')

amz_dict={"item_name":'',"item_price":'',"item_rating":'',"link_to_item":''}

try:
    amz_dict["item_name"]=new_soup1.find('span',attrs={"id":"productTitle"}).text.strip()[:50]
except Exception as e:
    amz_dict['item_name']=None
try:
    amz_dict["item_price"]=new_soup1.find('span',attrs={"class":"a-price aok-align-center reinventPricePriceToPayMargin priceToPay",'class':'a-offscreen'}).text
except Exception as e:
    amz_dict['item_price']=None
try:
    amz_dict["item_rating"]=new_soup1.find('span',attrs={'class':"reviewCountTextLinkedHistogram noUnderline"}).text.strip()
except Exception as e:
    amz_dict['item_raing']=None
amz_dict["link_to_item"]=amazon_result_url

print(amz_dict)

new_soup2=BeautifulSoup(new_webpage2.content,'html.parser')

flip_dict={"item_name":'',"item_price":'',"item_rating":'',"link_to_item":''}

try:
    flip_dict["item_name"]=new_soup2.find('span',attrs={"class":"B_NuCI"}).text.strip()[:50]
except Exception as e:
    flip_dict["item_name"]=None
try:
    flip_dict["item_price"]=new_soup2.find('div',attrs={"class":"_30jeq3 _16Jk6d"}).text
except Exception as e:
    flip_dict["item_price"]=None
try:
    flip_dict["item_rating"]=new_soup2.find('div',attrs={'class':"_3LWZlK"}).text.strip()
except Exception as e:
    flip_dict['item_rating']=None

flip_dict["link_to_item"]=flipkart_result_url

print(flip_dict)

